In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4),index=dates,columns=list('ABCD'))
df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ['E'])
df2 = pd.DataFrame({'A': 1.,
                    'B': pd.Timestamp('20130102'),
                    'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D': np.array([3] * 4, dtype='int32'),
                    'E': pd.Categorical(["test", "train", "test", "train"]),
                    'F': 'foo'})
s = pd.Series([1,3,5,np.nan,6,8],index=dates).shift(2)

In [6]:
df

,A,B,C,D
2013-01-01,-0.580407,-1.681435,1.338886,-1.695531
2013-01-02,-1.212343,0.810087,-0.883083,-0.158433
2013-01-03,-0.043790,2.915493,0.965412,0.074694
2013-01-04,-1.150028,-0.127283,0.996294,1.885400
2013-01-05,-0.369467,0.714230,-0.863941,0.324629
2013-01-06,-1.307036,-0.885400,-0.445092,-1.969323


#### Comparing array-like objects

You can conveniently perform element-wise comparisons when comparing a pandas data structure with a scalar value:

In [8]:
pd.Series(['foo','bar','baz']) == 'foo'

0     True
1    False
2    False
dtype: bool

In [9]:
pd.Index(['foo','bar','baz']) == 'foo'

array([ True, False, False])

Pandas also handles element-wise comparisons between different array-like objects of the same length:

In [10]:
pd.Series(['foo','bar','baz']) == pd.Index(['foo','bar','qux'])

0     True
1     True
2    False
dtype: bool

In [11]:
pd.Series(['foo','bar','baz']) == np.array(['foo','bar','qux'])

0     True
1     True
2    False
dtype: bool

Trying to compare Index or Series objects of different lengths will raise a ValueError:

In [12]:
np.array([1,2,3]) == np.array([2])

array([False,  True, False])

or it can return False if broadcasting can not be done:

In [13]:
np.array([1,2,3]) == np.array([1,2])

/Users/chitranjan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

### Combining overlapping data sets

In [14]:
df1 = pd.DataFrame({
    'A':[1.,np.nan,3.,5.,np.nan],
    'B':[np.nan,2.,3.,np.nan,6.]
})

In [15]:
df2 = pd.DataFrame({
    'A':[5.,2.,4.,np.nan,3.,7.],
    'B':[np.nan,np.nan,3.,4.,6.,8.]
})

In [16]:
df1

,A,B
0,1.0,NaN
1,NaN,2.0
2,3.0,3.0
3,5.0,NaN
4,NaN,6.0


In [17]:
df2

,A,B
0,5.0,NaN
1,2.0,NaN
2,4.0,3.0
3,NaN,4.0
4,3.0,6.0
5,7.0,8.0


In [18]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


### General DataFrame combine

The combine_first() method above calls the more general DataFrame.combine(). This method takes another DataFrame and a combiner function, aligns the input DataFrame and then passes the combiner function pairs of Series (i.e., columns whose names are the same).

So, for instance, to reproduce combine_first() as above:

In [19]:
def combiner(x,y):
    return np.where(pd.isna(x),y,x)

### 3.3.5 Descriptive statistics

There exists a large number of methods for computing descriptive statistics and other related operations on Series, DataFrame. Most of these are aggregations (hence producing a lower-dimensional result) like sum(), mean(), and quantile(), but some of them, like cumsum() and cumprod(), produce an object of the same size. Generally speaking, these methods take an axis argument, just like ndarray.{sum, std, }, but the axis can be specified by name or integer:


• Series: no axis argument needed
• DataFrame: index (axis=0, default), columns (axis=1) 

For example:

In [20]:
df

,A,B,C,D
2013-01-01,-0.580407,-1.681435,1.338886,-1.695531
2013-01-02,-1.212343,0.810087,-0.883083,-0.158433
2013-01-03,-0.043790,2.915493,0.965412,0.074694
2013-01-04,-1.150028,-0.127283,0.996294,1.885400
2013-01-05,-0.369467,0.714230,-0.863941,0.324629
2013-01-06,-1.307036,-0.885400,-0.445092,-1.969323


In [21]:
df.mean(0)

A   -0.777178
B    0.290949
C    0.184746
D   -0.256427
dtype: float64

In [22]:
df.mean(1)

2013-01-01   -0.654622
2013-01-02   -0.360943
2013-01-03    0.977953
2013-01-04    0.401096
2013-01-05   -0.048638
2013-01-06   -1.151713
Freq: D, dtype: float64

All such methods have a skipna option signaling whether to exclude missing data (True by default):

In [23]:
df.sum(0,skipna=False)

A   -4.663070
B    1.745691
C    1.108476
D   -1.538564
dtype: float64

In [24]:
df.sum(axis=1,skipna=True)

2013-01-01   -2.618487
2013-01-02   -1.443771
2013-01-03    3.911810
2013-01-04    1.604382
2013-01-05   -0.194550
2013-01-06   -4.606852
Freq: D, dtype: float64

Combined with the broadcasting / arithmetic behavior, one can describe various statistical procedures, like standard- ization (rendering data zero mean and standard deviation 1), very concisely:

In [25]:
ts_stand = ( df - df.mean() ) / df.std()

In [26]:
ts_stand.std()

A    1.0
B    1.0
C    1.0
D    1.0
dtype: float64

In [28]:
xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)

In [29]:
xs_stand.std(1)

2013-01-01    1.0
2013-01-02    1.0
2013-01-03    1.0
2013-01-04    1.0
2013-01-05    1.0
2013-01-06    1.0
Freq: D, dtype: float64

Note that methods like cumsum() and cumprod() preserve the location of NaN values. This is somewhat different from expanding() and rolling(). For more details please see this note.

In [30]:
df.cumsum()

,A,B,C,D
2013-01-01,-0.580407,-1.681435,1.338886,-1.695531
2013-01-02,-1.792750,-0.871348,0.455803,-1.853963
2013-01-03,-1.836539,2.044145,1.421216,-1.779269
2013-01-04,-2.986567,1.916862,2.417509,0.106131
2013-01-05,-3.356034,2.631092,1.553568,0.430759
2013-01-06,-4.663070,1.745691,1.108476,-1.538564


Here is a quick reference summary table of common functions. Each also takes an optional level parameter which applies only if the object has a hierarchical index.

Note that by chance some NumPy methods, like mean, std, and sum, will exclude NAs on Series input by default:

In [32]:
np.mean(df['A'])

-0.7771783977404798

In [34]:
np.mean(df['A'].to_numpy())

-0.7771783977404798

Series.nunique() will return the number of unique non-NA values in a Series:

In [35]:
series = pd.Series(np.random.randn(500))

In [36]:
series[20:500] = np.nan

In [37]:
series[10:20] = 5

In [38]:
series.nunique()

11

### Summarizing data: describe

There is a convenient describe() function which computes a variety of summary statistics about a Series or the columns of a DataFrame (excluding NAs of course):

In [39]:
series = pd.Series(np.random.randn(1000))

In [40]:
series[::2] = np.nan

In [41]:
series.describe()

count    500.000000
mean       0.021956
std        0.957585
min       -3.015134
25%       -0.593110
50%        0.050342
75%        0.649822
max        2.942714
dtype: float64

In [44]:
frame = pd.DataFrame(np.random.randn(1000,5),columns=['a','b','c','d','e'])

In [45]:
frame.iloc[::2] = np.nan

In [46]:
frame.describe()

,a,b,c,d,e
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,-0.035739,0.061744,0.002215,-0.026693,-0.019077
std,0.952523,1.039449,1.016104,0.999068,1.029850
min,-2.870968,-3.269927,-2.911425,-2.453823,-2.634322
25%,-0.699435,-0.577695,-0.685504,-0.750976,-0.677547
50%,-0.076752,0.053241,-0.054535,-0.027162,-0.081331
75%,0.595129,0.801656,0.744086,0.575257,0.705420
max,2.842469,3.097730,2.943138,2.765271,3.349105


You can select specific percentiles to include in the output:

In [47]:
series.describe(percentiles=[.05,.25,.75,.85])

count    500.000000
mean       0.021956
std        0.957585
min       -3.015134
5%        -1.595899
25%       -0.593110
50%        0.050342
75%        0.649822
85%        0.976575
max        2.942714
dtype: float64

By default, the median is always included.

For a non-numerical Series object, describe() will give a simple summary of the number of unique values and most frequently occurring values:

In [ ]:
s = pd.Series(['a','a','b','b','a','a',])